In [35]:
import numpy as np
import random

In [36]:
class data_preprocessing:
    def __init__(self,instance_path):
        self.instance_path=instance_path
        
        self.info, self.flights = self.read_file(f_name=self.instance_path)
        self.number_of_areas,self.starting_airport=int(self.info[0][0]),self.info[0][1]
        
        
        self.flights_by_day_dict = self.flights_by_day(flight_list=self.flights)
        self.flights_by_day_dict=self.redistribute_day_zero(data=self.flights_by_day_dict,
                                                            number_of_days=self.number_of_areas)
        
        self.flights_by_day_dict=self.remove_duplicate(flights_by_day=self.flights_by_day_dict)
        
        self.list_days= self.flights_by_day_dict.keys()
        
        self.airports_by_area = self.get_airports_by_areas()
        self.area_by_airport=self.invert_dict(original_dict=self.airports_by_area)
        
        self.starting_area=self.associated_area_to_airport(airport=self.starting_airport)
        self.list_airports=self.get_list_of_airports()
        self.list_areas=list(self.airports_by_area.keys())
        self.areas_connections_by_day=self.possible_flights_from_zone_to_zone_specific_day()
        
    def read_file(self,f_name):
        dist = []
        line_nu = -1
        with open(f_name) as infile:
            for line in infile:
                line_nu += 1
                if line_nu == 0:
                    index = int(line.split()[0]) * 2 + 1
                if line_nu >= index:
                    temp = line.split()
                    temp[2] = int(temp[2])
                    temp[3] = int(temp[3])
                    dist.append(temp)
                else:
                    dist.append(line.split())
            info = dist[:int(dist[0][0])*2+1]
            flights = dist[int(dist[0][0])*2+1:]
        return info, flights
    
    def flights_by_day(self,flight_list):
        # Create an empty dictionary to hold flights organized by day
        flights_by_day = {}

        # Iterate over each flight in the input list
        for flight in flight_list:
            # Extract the day from the flight entry
            day = flight[2]

            # Create a flight entry without the day
            flight_without_day = flight[:2] + flight[3:]

            # Add the flight to the corresponding day in the dictionary
            if day not in flights_by_day:
                flights_by_day[day] = []
            flights_by_day[day].append(flight_without_day)
            
        return flights_by_day
    
    def redistribute_day_zero(self,data, number_of_days):
        # Check if the dictionary has only key 0
        if len(data) == 1 and 0 in data:
            # Get the values from key 0
            zero_values = data[0]
            
            # Create a new dictionary with keys from 1 to number_of_days
            new_data = {i: zero_values.copy() for i in range(1, number_of_days + 1)}
            
            return new_data
        else:
            # If key 0 exists and there are other keys
            if 0 in data:
                # Get the values from key 0
                zero_values = data[0]
                
                # Iterate over all other keys
                for key in data.keys():
                    if key != 0:
                        # Add the values from key 0 to each of the other keys
                        data[key] += zero_values
                        
                # Remove key 0 from the dictionary
                del data[0]
            
            return data
    
    def flights_from_airport(self,flights_by_day, from_airport, considered_day):
        flights_from_airport = []
        for day, flights in flights_by_day.items():
            if day==considered_day:
                for flight in flights:
                    if flight[0] == from_airport:
                        flights_from_airport.append(flight)
                return flights_from_airport
            else:
                return None

    def invert_dict(self,original_dict):
        inverted_dict = {}
        for key, value_list in original_dict.items():
            for value in value_list:
                if value in inverted_dict:
                    inverted_dict[value].append(key)
                else:
                    inverted_dict[value] = key
        return inverted_dict

    def get_cost(self, day, from_airport, to_airport):
        flights = self.flights_by_day_dict.get(day, [])
        return next(
            (
                flight[2]
                for flight in flights
                if flight[0] == from_airport and flight[1] == to_airport
            ),
            float('inf'),
        )

    def possible_flights_from_zone_to_zone_specific_day(self):
        areas_connections_by_day = {}

        for day, flights in self.flights_by_day_dict.items():
            areas_connections_list = []

            for flight in flights:
                connection = f"{self.area_by_airport.get(flight[0])} to {self.area_by_airport.get(flight[1])}"
                if connection not in areas_connections_list:
                    areas_connections_list.append(connection)

            areas_connections_by_day[day] = areas_connections_list

        return areas_connections_by_day

    def get_airports_by_areas(self):
        area_num = int(self.info[0][0])
        return {f"{i}": self.info[2+i * 2] for i in range(0, area_num)}
    
    def get_list_of_airports(self):
        unique_airports = set()

        # Iterate through each sublist and add elements to the set
        for sublist in self.airports_by_area.values():
            for airport in sublist:
                unique_airports.add(airport)
        
        return list(unique_airports)
                    
    def associated_area_to_airport(self,airport):
        return next(
            (
                area
                for area, airports in self.airports_by_area.items()
                if airport in airports
            ),
            "Airport not found",
        ) 
    
    def remove_duplicate(self,flights_by_day):
        for day, flights in flights_by_day.items():
            unique_flights = {}
            for flight in flights:
                flight_key = (flight[0], flight[1])
                if flight_key not in unique_flights:
                    unique_flights[flight_key] = flight
                else:
                    if flight[2] < unique_flights[flight_key][2]:
                        #print(flight[2],unique_flights[flight_key][2])
                        unique_flights[flight_key] = flight
                flights_by_day[day] = list(unique_flights.values())
        return flights_by_day
    
    def possible_flights_from_an_airport_at_a_specific_day(self,day,from_airport):
        daily_flights = self.flights_by_day_dict.get(day, [])
        
        flights_from_airport = []
        for flight in daily_flights:
            if flight[0] == from_airport:
                
                flights_from_airport.append([flight[1], flight[2]])

        return flights_from_airport

In [166]:
Data_Preprocessing=data_preprocessing(instance_path="Flight connections dataset/5.in")

In [135]:
class heuristic_operators:
    def __init__(self):
        pass

    @staticmethod
    def swap(arr, a, b):
        arr[a],arr[b] = arr[b], arr[a]

    @staticmethod
    def reverse(arr, a, b):
        if a > b:
            a, b = b, a
        arr[a:b+1] = arr[a:b+1][::-1]

    @staticmethod
    def insert(arr, a, b):
        temp = arr[a]
        del arr[a]
        arr.insert(b, temp)

    @staticmethod
    def swap_k(arr, a, areas_n):
        s = np.random.randint(1, areas_n, size=a * 2)
        for i in range((a // 2) + 2):
            temp = arr[s[i]]
            arr[s[i]] = arr[s[i + 1]]
            arr[s[i + 1]] = temp
            
    @staticmethod
    def swap_target(arr, from_index, to_index):

        if from_index >= len(arr) or to_index >= len(arr):
            raise IndexError("Target or position is out of range")
        
        arr[from_index], arr[to_index] = arr[to_index], arr[from_index]

In [169]:
class heuristics:
    def __init__(self, data_preprocessing_class):
        self.data = data_preprocessing_class
        
        self.starting_airport = self.data.starting_airport
        self.starting_area=self.data.starting_area
        self.total_cost = 0
        
        self.feasible_airports_solutions=self.create_feasible_area_solutions(number_solutions=10,number_iterations=1000)
        #self.candidates_area_solution=self.find_list_initial_area_solutions(number_instances=2)
        #self.check_feasibility_airports(airport_solution=self.airport_initial_solution)
        
        #self.find_initial_solution()
    
    def cost(self, airport_solution):
        total_cost = 0
        for day, (from_airport, to_airport) in enumerate(zip(airport_solution, airport_solution[1:]), start=min(self.data.list_days)):
            total_cost += self.data.get_cost(day, from_airport, to_airport)
        return total_cost

    def check_feasibility_area(self, area_solution):
        # sourcery skip: assign-if-exp, boolean-if-exp-identity, reintroduce-else, remove-unnecessary-cast
        
        if area_solution[0]!=area_solution[-1]:
            return False
        
        # Ensure each area is visited exactly once, except the starting/ending area
        area_counts = {area: area_solution.count(area) for area in set(area_solution)}
        
        # Check the counts
        if area_counts[self.starting_area] != 2:
            return False
        for area in self.data.list_areas:
            if area != self.starting_area and area_counts.get(area, 0) != 1:
                return False
            
        area_solution_feasibility=[]
        for i in range(len(area_solution)-1):
            day = i + min(self.data.list_days)
            #print(f"DAY {day}")

            connection = f"{area_solution[i]} to {area_solution[i + 1]}"
            
            if connection in self.data.areas_connections_by_day.get(day, []):
                area_solution_feasibility.append(True)
            else:
                area_solution_feasibility.append(False)

        #print(area_solution_feasibility)
        if False in area_solution_feasibility:
            return False
        
        return True
        
    def check_feasibility_airports(self,airport_solution):
        airport_solution_feasibility=airport_solution.copy()
        airport_solution_feasibility={}

        # Iterate over the range of the length of the airport solution minus 1
        for k in range(len(airport_solution)-1):
            # Calculate the cost and assign it to the dictionary with key k
            airport_solution_feasibility[k] = self.data.get_cost(
                day=k + min(self.data.list_days),
                from_airport=airport_solution[k],
                to_airport=airport_solution[k + 1]
            )

        for k in airport_solution_feasibility:
            if airport_solution_feasibility[k]==False:
                return False
        
        return True

    def find_initial_solution(self):
        initial_area_solution=self.area_initial_solution.copy()
        initial_airport_solution=self.airport_initial_solution.copy()
        
        while self.check_feasibility_area(area_solution=initial_area_solution)!=True:
            x=np.random.randint(1,len(initial_area_solution)-1)
            y=np.random.randint(1,len(initial_area_solution)-1)
            
            heuristic_operators.swap_target(arr=initial_area_solution,
                                            from_index=x,
                                            to_index=y)
            
        heuristic_operators.swap_target(arr=initial_airport_solution,
                                            from_index=x,
                                            to_index=y)
            
        initial_airport_solution[x]=random.choice(self.data.airports_by_area.get(self.area_initial_solution[x]))

        self.area_initial_solution=initial_area_solution
        self.airport_initial_solution=initial_airport_solution
            
    def create_feasible_area_solutions(self,number_solutions,number_iterations):
        area_initial_solution=self.data.list_areas.copy()
        
        for k in range(len(area_initial_solution)):
            if area_initial_solution[k]==self.starting_area:
                heuristic_operators.swap_target(arr=area_initial_solution,
                                                from_index=k,
                                                to_index=0)
        area_initial_solution.append(self.starting_area)
        
        
        list_solutions=[]
        
        if self.check_feasibility_area(area_initial_solution):
            list_solutions.append(area_initial_solution)
            
        for _ in range(number_iterations):
            x=np.random.randint(1,len(area_initial_solution)-1)
            y=np.random.randint(1,len(area_initial_solution)-1)
            potential_solution=area_initial_solution.copy()
            heuristic_operators.swap_target(potential_solution,from_index=x,to_index=y)
            if self.check_feasibility_area(area_solution=potential_solution) and potential_solution not in list_solutions:
                list_solutions.append(potential_solution)
                #print(potential_solution)
                  
        return list_solutions
        
    def find_list_initial_area_solutions(self, number_instances):
        return [
            self.create_initial_random_solution() for _ in range(number_instances)
        ]
    

In [167]:
Heuristics = heuristics(data_preprocessing_class=Data_Preprocessing)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '0']


In [168]:
len(Heuristics.feasible_airports_solutions)

613

In [107]:
Heuristics.check_feasibility_area(S)

True

In [93]:
Heuristics.area_initial_solution

[]

In [83]:
Heuristics.area_initial_solution

[['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '0'],
 ['0',
  '1',
  '2',
  '3',
  '33',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '4',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '0'],
 ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '37',
  '31',
  '32',
  '33',
  '34'

In [49]:
a=[0,1,2,3,4]

In [ ]:
heuristic_operators.swap_target(arr=a,
                                from_index=random.randint(0,len(a)-1),
                                to_index=

In [47]:
Heuristics.candidates_area_solution[0]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '0']

In [42]:
print(Heuristics.area_initial_solution)
print(Heuristics.check_feasibility_area(area_solution=Heuristics.area_initial_solution))
print(Heuristics.airport_initial_solution)
print(Heuristics.check_feasibility_airports(airport_solution=Heuristics.airport_initial_solution))
print(Heuristics.cost(airport_solution=Heuristics.airport_initial_solution))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '0']
True


AttributeError: 'heuristics' object has no attribute 'airport_initial_solution'